## DenseNet
- Paper : (2018)_DenseNet-Densely Connected Convolutional Networks
- 진화된 Skip connection(shortcut connection)과 Bottleneck layers를 적용
    - 각 층은 모든 앞 단계에서 올 수 있는 지름길 연결 구성 (모든 층은 연결되어 있다)
    - Element-wise addition !
- ReseNet의 아이디어를 계승함
    - 깊은 레이어를 쌓을 수록 기울기 소실 문제 해결 및 Feature 추출
- 각각의 layer는 이전 모든 layer들로 부터 추가적인 input을 받으며, 현재의 feature map을 다음 layer들에 전달
- 각각의 layer들은 loss function과 input signal로 부터의 gradient에 direct하게 접근하여, 더 쉽게 학습 가능

![densenet.png](img/densenet.png)

## DenseNet 장점
- 기울기 소실 완화 (alleviate the vanishing gradient)
- 전방향 강화 (strengthen feature propagation, information is preserved)
- 피쳐 재사용 강화 (encourage feature reuse)
- 파라미터 수를 줄임 (substantially reduce the number of parameters, fewer parameter)

<hr>


## Concatenation 과정

![densenet_forward](img/densenet_forward.PNG)

- 빨강색
- 초록색 = 빨간색 + 초록색(base)
- 보라색 = 빨간색 + 초록색 + 보라색(base)
- 노랑색 = 빨간색 + 초록색 + 보라색 + 노란색(base)
- 갈색 = 빨간색 + 초록색 + 보라색 + 노란색 + 갈색(base)



<hr>

## DenseNet 전체 과정

![densenet_all](img/densenet_all.png)

<hr>

## DenseNet 큰 특징
- **DenseNet Block + Transition Layer**
- 각 층은 입력 특징맵과 같은 차원의 특징 지도 생성
- 밀집 블록(Dense block)과 전이층 (transition layer = 1x1 conv + maxpool)으로 구성
    - 병목층 : 1x1 컨볼루션, 출력되는 특징맵의 채널 수 측소
    - 병목층이 있는 층 ( BN -> ReLU -> 1x1 conv -> BN -> ReLU -> 3x3 conv)
    - 특징맵의 크기를 줄이기 위한 풀링 연산 적용 필요


![densenet_structure](img/densenet_structure.PNG)

- 1. DenseNet Block
    - 1) **Dense Block Main**
        - BN -> ReLU -> 3x3 Conv 구조
        - 연산이 급격하게 증가하는 것을 막기 위한 Bottleneck 구조 적용
        - BN -> ReLU -> 1x1Conv -> BN -> ReLU -> 3x3Conv 구조로 변경 하여 계산 복잡성을 줄여줌
    
    - 2) **Dense Block Pre-Activation 구조**
        - **Preactivation = BatchNorm - ReLU - Conv 과정**을 거침
        - ResNet과 같이 Pre-Activation 구조를 사용
        
    - 3) **Dense Block Growth rate(k)**
        - ex) growth rate k = 4 이면 3x3x c x4 / 3x3x(c+4)x4 / 3x3x(c+8)x4 / 3x3x(c+12)x4
        - L번째 layer는 k0 + k*(l-1) 개의 feature map 을 가짐
        - 이전 특징 맵에 누적해서 Conacatenate 하는 것
        - 이런 다 더하는 것 때문에 파라미터가 많을 것 같지만, K값이 작아도 가능해서 그렇게 많지 않음
        - growth rate 값이 클경우 진행하다가 1x1로 conv로 한번 연산량을 줄여주고 다시 3x3 진행

<br>

- 2. Transition layer (Convolution + Pooling layers)
    - **다운샘플링 : Feature map의 가로, 세로 사이즈를 줄여주고 feature map의 갯수를 줄여줌**
        - 다음 Dense block의 Feature map 갯수가 달라지기 위한 Conv 및 Pooling
        - Pooling layer들은 Denseblock사이의 downsampling 하는 역할로 사용
    - Concatenation은 feature map 사이즈가 다르면 연산이 불가능 
    - **1x1 Convloution layer + 2x2 average pooling layer**

- 3. Classification Layer
    - Global Average Pooling + FC (linear) layer
    
<hr>




## Difference Between Resnet and Densenet
- **둘다 Bottleneck 구조를 가짐** 

- ResNet
    - ResNet은 이전의 정보를 합하는 identity function을 사용
    - Element-wise addition
    - Resnet은 (1x1, 64) 으로 dimension reduction + (1x1, 256) expand dimension을 과정을 거침

- DenseNet
    - Channel-wise concatenation
    - DenseNet은 여러 network를 concatenation을 사용하여 정보를 더 보존
    
    - Densenet은 1x1, 4*k 로 줄이기만함. 단 concatenate를 이용한 등차수열적인 이용

![resnet_vs_densenet](img/resnet_vs_densenet.PNG)

<hr>

## Difference Between Resnet and Densenet Parameter

![resnet_vs_densenet_parameter](img/resnet_vs_densenet_parameter.PNG)


![densenet_block](img/densenet_block.png)


<hr>

## 실험결과 

### Cifar-10
![densenet_result](img/densenet_result.PNG)

### Cifar-100
![densenet_result_cifar100](img/densenet_result_cifar100.PNG)

### ImageNet
![densenet_result_Imagenet](img/densenet_result_Imagenet.PNG)


## 참고문헌
- https://warm-uk.tistory.com/46#:~:text=DenseNet%EC%9D%80%20extremly%20deep%20%ED%95%98%EA%B1%B0%EB%82%98,%EA%B0%80%EC%A7%84%20%EC%95%95%EC%B6%95%20%EB%AA%A8%EB%8D%B8%EC%9D%84%20%EB%A7%8C%EB%93%A0%EB%8B%A4.
- https://www.youtube.com/watch?v=fe2Vn0mwALI&t=347s // PR-028
- https://ysbsb.github.io/cnn/2020/02/12/DenseNet.html
- https://pdfs.semanticscholar.org/c3d9/26a85d85a83126f405ad40ff453611148c15.pdf